# 🛍️ ShopSmart.AI – An Intelligent, Voice-Activated Shopping Advisor using Gen AI & CrewAI

## 📌 Problem Statement
Modern online shopping platforms often overwhelm users with too many options. Users need to switch between platforms, manually compare product features, read countless reviews, and still struggle to make confident decisions. This is time-consuming and frustrating.

**What if** we could just tell an AI agent what we want, and it would:
- Understand our preferences
- Search across multiple platforms
- Analyze product listings and reviews
- Recommend the best product
- Show results in a structured format like JSON or table

That’s exactly what **ShopSmart.AI** solves.

---

## 💡 Project Objective
Build a multi-agent AI-powered shopping assistant using:
- **CrewAI** for agent collaboration
- **Google GenAI (Gemini)** for LLM-based reasoning
- **Serper API** for real-time web search
- **RAG (Retrieval-Augmented Generation)** for summarizing product reviews

---

## 🛠️ Technologies, Tools & Gen AI Capabilities Mapping (Updated)

| Tool / Library                 | Purpose                                                                 | Gen AI Capability Demonstrated                         |
|-------------------------------|-------------------------------------------------------------------------|--------------------------------------------------------|
| `CrewAI`                      | Manages agents and orchestrates multi-step processes                    | ✅ Multi-Agent Collaboration                           |
| `SerperDevTool`               | Searches the web for product listings                                   | ✅ Information Retrieval for RAG                       |
| `ScrapeWebsiteTool`           | Scrapes data from product listing websites                              | ✅ Data Collection for downstream processing           |
| `WebsiteSearchTool + RAG`     | Retrieves and summarizes user reviews                                   | ✅ Retrieval-Augmented Generation (RAG)                |
| `Google Gemini LLM`           | Provides reasoning, analysis, and natural language output               | ✅ Structured Output / JSON Mode<br>✅ Review Summarization |
| `LiteLLM`                     | Runs Gemini LLMs efficiently                                            | ✅ Efficient LLM Inference                             |
| `Google Embeddings`           | Used for document retrieval in RAG setup                                | ✅ Vector Search for RAG                               |
| `Colab + Python`              | Development environment                                                 | ✅ Experimentation and Prototyping                     |
| `speech_recognition`          | Converts user voice into text                                           | ✅ Voice-to-Text Input (Integrated!)                   |

---

### 🔊 New Feature Added

**Voice Input Powered by `speech_recognition`:**  
Users can now *speak* their shopping queries instead of typing. The spoken query is transcribed and seamlessly passed to the input agent for further processing.

---


### Install depedencies and import Libraries

In [1]:
!pip install crewai crewai_tools litellm -q
!pip install google-generativeai>=0.8.4 -q
!pip install python-dotenv -q
!pip install SpeechRecognition -q

  You can safely remove it manually.


In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [4]:
from crewai import Agent,LLM
from dotenv import load_dotenv
from crewai import Agent, Crew, Process, Task, LLM
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
import os

from crewai import Agent, Crew, Process, Task, Flow
from crewai.project import CrewBase, agent, crew, task

from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import speech_recognition as sr

### Get APi KEys and 🧠 Load Gemini LLM

In [5]:
#from google.colab import userdata

# Access the secrets using their correct names
#SERPER_API_KEY = userdata.get("SERPER_API_KEY")
#GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")

In [6]:
load_dotenv(".env")

SERPER_API_KEY = os.getenv("SERPER_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [19]:
#print(SERPER_API_KEY)
#print(GOOGLE_API_KEY)

In [7]:
gemini_llm = LLM(
    api_key=GOOGLE_API_KEY,
    model="gemini/gemini-2.0-flash-lite",
    temperature=0,
    max_tokens=None
)

## 🔍 How It Works – Step-by-Step

1. **User Input Collector Agent**  
   - Gathers user preferences (product type, budget, features) via **text** or **voice** input

2. **Web Search Specialist Agent**  
   - Uses **Serper API** to fetch live product data from e-commerce platforms (Google, Amazon, Daraz).

3. **Product Analyst Agent**  
   - Compares product specs, prices, and features, highlighting top options.

4. **Review Analyzer Agent**  
   - Summarizes product reviews using **RAG**, providing pros/cons and sentiment analysis.

5. **Shopping Recommendation Agent**  
   - Recommends the best product based on analysis, user preferences, and reviews.

---


In [8]:
#helper function for voice inpout

# Helper function for voice input
def get_voice_input():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("🎙️ Listening for voice input... Please speak.")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio)
            st.success("Recognized: " + text)
            return text
        except sr.UnknownValueError:
            st.error("❌ Could not understand audio.")
            return None
        except sr.RequestError as e:
            st.error(f"❌ Request failed: {e}")
            return None




In [9]:
# Define Agents
input_collector = Agent(
    role="User Input Collector",
    goal="Gather and clarify user requirements for product search from text or voice input, with a focus on products available in Pakistan.",
    backstory="You are an expert in understanding user needs from various input types and translating them into clear search parameters, ensuring the results are relevant to users in Pakistan.",
    llm=gemini_llm,
    verbose=True
)

# Define  web search Tools
# Tools for specific websites
search_tool = SerperDevTool(api_key=SERPER_API_KEY)

scrape_google = ScrapeWebsiteTool(website_url='https://google.com/')
scrape_amazon = ScrapeWebsiteTool(website_url='https://www.amazon.com/')
scrape_daraz = ScrapeWebsiteTool(website_url='https://www.daraz.pk/')


web_searcher = Agent(
    role="Web Search Specialist",
    goal="Find product listings across Google, Amazon, and Daraz",
    backstory="You are a skilled product search expert who knows how to extract valuable listings from multiple platforms.",
    tools=[search_tool, scrape_google, scrape_amazon, scrape_daraz],
    llm=gemini_llm,
    allow_delegation=False,
    verbose=True
)



analyst = Agent(
    role="Product Comparison Expert",
    goal="Evaluate and compare product listings from different vendors to identify the best option based on quality, price, and reviews.",
    backstory="You are a skilled product analysis expert. Your job is to compare various options available online, understand their features and pricing, and help users make informed decisions by providing pros, cons, and recommendations.",
    llm=gemini_llm,
    verbose=True
)

review_tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="google",
            config=dict(
                model="gemini/gemini-2.0-flash-lite",
                api_key=GOOGLE_API_KEY
            )
        ),
        embedder=dict(
            provider="google",
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document"
            )
        )
    )
)

review_agent = Agent(
    role="Review Analyzer",
    goal="Analyze reviews and summarize user sentiment for the top product recommended by the analysis task from the specified vendor.",
    backstory="You analyze reviews from the selected vendor's website (Daraz, Amazon, AliExpress) for the chosen product to extract pros, cons, and overall user sentiment.",
    tools=[review_tool],
    llm=gemini_llm,
    verbose=True
)

# Define the final recommendation agent
recommender = Agent(
    role="Shopping Recommendation Specialist",
    goal="Recommend the best product based on analysis and user preferences",
    backstory="You provide tailored product recommendations with clear reasoning, combining product details and customer reviews to help users make the best purchase decision.",
    llm=gemini_llm,
    verbose=True
)

### 🛠️ Task Flow 

1. **📥 Input Task: User Preferences to Refined Query**
   - Collects user preferences:  
     - **Product Type**, **Use Case**, **Preferred Brands**, **Features**, **Budget**, **Condition**  
   - Outputs a **structured search query** for product search.

---

2. **🔍 Search Task: Online Product Search**
   - Searches e-commerce platforms (e.g., Amazon, Daraz) using the refined query.
   - Retrieves product details:
     - Name, Price, Seller Info, Product Link

---

3. **📊 Analysis Task: Product Comparison**
   - Compares products based on:
     - **Price**, **Ratings**, **Features**  
   - Outputs a **summary** with pros, cons, price range, and feature comparison.

---

4. **📝 Review Task: Summarize Reviews**
   - Uses **RAG (Retrieval-Augmented Generation)** to analyze reviews.
   - Outputs **bullet-point pros and cons** for each product.

---

5. **✅ Recommendation Task: Final Product Suggestion**
   - Recommends the **best product** based on analysis and reviews.
   - Includes:
     - Product Name, Price, Key Features, Reason for Recommendation, Purchase Link


In [10]:
# Define Tasks
filters = {
    "min_rating": 4.0,
    "brand": "Sony"
}

# Manually format the filters
brand = filters["brand"] if filters["brand"] else "Any"

description = (
    f"Process the user input: '{{user_input}}'\n"
    f"Use the following filters if applicable:\n"
    f"- Minimum Rating: {filters['min_rating']}\n"
    f"- Preferred Brand: {brand}\n"
    "Generate a refined product search query based on these inputs."
)

# Now, use the formatted description in your Task
input_task = Task(
    description=description,
    expected_output="A well-formed product search query based on the user's input.",
    agent=input_collector
)


search_task = Task(
    description="""
        Search online for the best matching products using the refined search query.
        Look for product listings across Google, Amazon, and Daraz. Use appropriate tools for each platform 
        (e.g., Serper API for Google, scraping for Amazon and Daraz). Return a summarized list with key details 
        like title, price, link, and brief description.
    """,
    expected_output="A list of product listings from Google, Amazon, and Daraz with key details.",
    agent=web_searcher,
    context=[input_task]
)

analysis_task = Task(
    description=(
        "Analyze the provided product listings from different websites (Google, Amazon, Daraz). "
        "Compare key features, prices, availability, and customer ratings. "
        "Summarize the pros and cons of each option and select the best product(s) based on overall value. "
        "Include the vendor name (Amazon, Daraz, or AliExpress) for the top recommended product."
    ),
    expected_output="A summary of pros and cons for each product, with a final recommended option, reasoning, and the vendor name.",
    agent=analyst,
    context=[search_task]
)


review_task = Task(
    description=(
        "Using the top product recommendation and vendor from the analysis task, "
        "summarize customer reviews for this product. Review feedback will include pros, cons, and sentiment analysis. "
        "First, determine the correct website URL based on the vendor (Amazon, Daraz, or AliExpress). "
        "Then use the WebsiteSearchTool to analyze reviews from that specific website."
    ),
    expected_output="A summarized list of pros, cons, and user sentiment for the selected product from the appropriate vendor's website.",
    agent=review_agent,
    context=[analysis_task]
)

recommendation_task = Task(
    description="Provide the best product recommendation based on features, customer reviews, and user preferences.",
    expected_output="A concise product recommendation with summarized reasoning, including product features, price, pros/cons, and customer sentiment.",
    agent=recommender,
    context=[
        analysis_task,  # Output from Analysis Agent (Best Product)
        review_task     # Output from Review Agent (Review Analysis)
    ]
)

In [11]:
product_knowledge = StringKnowledgeSource(
    content="Information about current product trends, including electronics, fashion, beauty, lifestyle, and more."
)

### 🧑‍🤝‍🧑 Assemble the Crew

In [12]:
# 🤖 Shopping Crew Setup
shopping_crew = Crew(
    agents=[input_collector, web_searcher, analyst, review_agent, recommender],
    tasks=[input_task, search_task, analysis_task, review_task, recommendation_task],
    verbose=True,
    process=Process.sequential,
    embedder={
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": GOOGLE_API_KEY,
        }
    }
)


# This will generate an interactive HTML plot of your flow
#shopping_crew.plot("shop_smart_flow.html")



In [13]:
# Replace Streamlit functions with standard Python input for testing in Jupyter

# Choose input type (simulate user choice between Text and Voice)
input_type = input("Choose input type (Text/Voice): ").strip()

# Get user input directly for text input
if input_type.lower() == "text":
    user_input = input("Please enter your product query: ")
elif input_type.lower() == "voice":
    # Simulate voice input (you can replace this with a real voice function for actual testing)
    user_input = "Simulated voice input: search for laptops"  # Replace this with actual voice input handling
else:
    user_input = None
    print("Invalid input type selected. Please try again.")

# If input is provided, process the input using Crew AI
if user_input:
    # Simulate calling the Crew AI and kicking off the tasks
    result = shopping_crew.kickoff(inputs={"user_input": user_input})

    # Output the result
    print("Processed result:", result.raw)
else:
    print("No input received. Please try again.")


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 454ba020-63e3-47f5-a371-2a826c989f52                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 9f170ad6-1f45-4d30-91b2-71b7c61a69c7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 9f170ad6-1f45-4d30-91b2-71b7c61a69c7
       Status: Executing Task...
    └── 🤖 Agent: User Input Collector
            Status: In Progress

# Agent: User Input Collector
## Task: Process the user input: 'help me find shoes under 5000 pkr for a party colour black'
Use the following filters if applicable:
- Minimum Rating: 4.0
- Preferred Brand: Sony
Generate a refined product search query based on these inputs.


🤖 Agent: User Input Collector
    Status: In Progress
└── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 9f170ad6-1f45-4d30-91b2-71b7c61a69c7
│      Assigned to: User Input Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: User Input Collector
│           Status: ✅ Completed
├── 📋 Task: e6c76808-7a02-4011-a1e8-5eddbbff5c09
│      Assigned to: Web Search Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Search Specialist
│           Status: ✅ Completed
├── 📋 Task: af17e91b-262c-4dfb-a0d8-4dfe97216d3f
│      Assigned to: Product Comparison Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Product Comparison Expert
│           Status: ✅ Completed
├── 📋 Task: 1441ae55-f389-4320-8ce8-e142d9ce40d9
│      Assigned to: Review Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Review Analyzer
│           Status: ✅ Completed
└── 📋 Task: 22737d68-31f9-429e-b35e-02e38845987d
       Status: Executing Task...
    └── 🤖 Agent: Shopping Recommendation Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 9f170ad6-1f45-4d30-91b2-71b7c61a69c7
│      Assigned to: User Input Collector
│      Status: ✅ Completed
│   └── 🤖 Agent: User Input Collector
│           Status: ✅ Completed
├── 📋 Task: e6c76808-7a02-4011-a1e8-5eddbbff5c09
│      Assigned to: Web Search Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Web Search Specialist
│           Status: ✅ Completed
├── 📋 Task: af17e91b-262c-4dfb-a0d8-4dfe97216d3f
│      Assigned to: Product Comparison Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Product Comparison Expert
│           Status: ✅ Completed
├── 📋 Task: 1441ae55-f389-4320-8ce8-e142d9ce40d9
│      Assigned to: Review Analyzer
│      Status: ✅ Completed
│   └── 🤖 Agent: Review Analyzer
│           Status: ✅ Completed
└── 📋 Task: 22737d68-31f9-429e-b35e-02e38845987d
       Assigned to: Shopping Recommendation Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Shopping Recommendation Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 22737d68-31f9-429e-b35e-02e38845987d                                                                     │
│  Agent: Shopping Recommendation Specialist                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 454ba020-63e3-47f5-a371-2a826c989f52                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Processed result: **Product Recommendation: Black Party Shoes (General)**

**Reasoning:**

Based on the provided vendor analysis, the best approach is to explore the websites of the following vendors, prioritizing those with a broader selection and a reputation for offering affordable options within the 5000 PKR budget:

1.  **Bata Pakistan:**
    *   **Features:** Likely to offer a variety of shoe styles, potentially including party-appropriate options. Known for affordability.
    *   **Price:** Potentially within the budget, especially during sales.
    *   **Pros:** High probability of finding affordable options.
    *   **Cons:** Specific styles and availability within the budget are unknown without checking their website.
    *   **Customer Sentiment:** Unable to provide customer reviews without website access.
    *   **Recommendation:** Visit the Bata Pakistan website and check the "Super Sale Men" section.

2.  **Ndure:**
    *   **Features:** Offers a wide range of men's shoe

### 📊 Sample Use Case
#### Input Query:
i used this search query :
"help me find party shoes under 5000 pkr ladies"

#### Output:

Structured JSON response showing product name, price, features, review summary, and a final recommendation with reasoning.

### 🚧 Limitations
Scraping dynamic websites can fail or be rate-limited.

Product availability may vary across regions.
## ✅ Conclusion and Deployment

This project demonstrates the power of **Generative AI + CrewAI** agents working collaboratively to create an intelligent, voice-activated product recommendation system tailored for users in Pakistan.  
It uses **Google Gemini APIs**, **Serper API**, and **Streamlit** as the frontend to deliver an interactive shopping experience powered by autonomous agents.

> 🔗 I have deployed this application on **Streamlit** for demonstration purposes as part of my **Capstone Project Video**.  
A **public Streamlit link** will be shared here soon for live testing and demo access.
------------------

---

📌 **If you found this notebook helpful, don't forget to _Upvote_ ❤️ and _Share_ it with others!**

🔗 Sharing is caring! Let's help more people explore and learn together.

---
